In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [2]:
df_original = pd.read_csv('../data/_data.csv')
df = df_original.copy()
df.head(2)

,Unnamed: 0,ID объявления,Количество комнат,Тип,Метро,Адрес,"Площадь, м2",Дом,Парковка,Цена,...,Окна,Санузел,Можно с детьми/животными,Дополнительно,Название ЖК,Серия дома,"Высота потолков, м",Лифт,Мусоропровод,Ссылка на объявление
0,0,271271157,4,Квартира,м. Смоленская (9 мин пешком),"Москва, улица Новый Арбат, 27",200.0/20.0,"5/16, Монолитный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",...,NaN,NaN,"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...","Новый Арбат, 2010",NaN,3.0,"Пасс (4), Груз (1)",Да,https://www.cian.ru/rent/flat/271271157
1,1,271634126,4,Квартира,м. Смоленская (8 мин пешком),"Москва, улица Новый Арбат, 27",198.0/95.0/18.0,"5/16, Монолитно-кирпичный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",...,На улицу и двор,"Совмещенный (2), Раздельный (1)",Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.5,"Пасс (1), Груз (1)",Нет,https://www.cian.ru/rent/flat/271634126


In [3]:
# 1. проверяем в каких колонках есть пропуски
df.isna().sum()

Unnamed: 0                      0
ID  объявления                  0
Количество комнат            1041
Тип                             0
Метро                        1315
Адрес                           0
Площадь, м2                     0
Дом                             0
Парковка                    13417
Цена                            0
Телефоны                        0
Описание                        0
Ремонт                       2755
Площадь комнат, м2           8910
Балкон                       7978
Окна                         6613
Санузел                      2672
Можно с детьми/животными     6096
Дополнительно                 357
Название ЖК                 17520
Серия дома                  21205
Высота потолков, м          12162
Лифт                         5500
Мусоропровод                10522
Ссылка на объявление            0
dtype: int64

In [4]:
# 23368 rows × 25 columns
# 1. заполняем пропуски:
# судя по пропускам можно удалить столбец "Серия дома" и "Название ЖК" 1
# "Количество комнат" удаляем выбросы заполняем медианой по общей площади (или удалить строки с нулями) 1
# "Метро" заполним 'unknown' (или удалить строки с нулями) 1
# "Парковка" удаляем выбросы заполняем моду или может 'unknown'
# "Ремонт" удаляем выбросы заполняем модой
# "Площадь комнат, м2" удаляем выбросы заполняем медианой по общей площади
# "Балкон" 'unknown'
# "Окна" удаляем выбросы заполняем модой
# "Санузел" удаляем выбросы заполняем модой ----
# "Можно с детьми/животными" заполним 'unknown'
# "Дополнительно" заполним 'unknown'
# "Название ЖК" удалим стобец
# "Серия дома" удалить столбец
# "Высота потолков" удаляем выбросы и заполняем медианой по высоте потолков
# "Лифт" 'unknown'
# "Мусоропровод" 'unknown'  может удалить столбец потому что по графику нет зависимости от цены

In [5]:
# 2. удаляем столбцы "Серия дома" и "Название ЖК"
df.drop(columns=['Серия дома', 'Название ЖК'], inplace=True)
df.head(2)

,Unnamed: 0,ID объявления,Количество комнат,Тип,Метро,Адрес,"Площадь, м2",Дом,Парковка,Цена,...,"Площадь комнат, м2",Балкон,Окна,Санузел,Можно с детьми/животными,Дополнительно,"Высота потолков, м",Лифт,Мусоропровод,Ссылка на объявление
0,0,271271157,4,Квартира,м. Смоленская (9 мин пешком),"Москва, улица Новый Арбат, 27",200.0/20.0,"5/16, Монолитный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",...,NaN,NaN,NaN,NaN,"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...",3.0,"Пасс (4), Груз (1)",Да,https://www.cian.ru/rent/flat/271271157
1,1,271634126,4,Квартира,м. Смоленская (8 мин пешком),"Москва, улица Новый Арбат, 27",198.0/95.0/18.0,"5/16, Монолитно-кирпичный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",...,25 25 20 25,NaN,На улицу и двор,"Совмещенный (2), Раздельный (1)",Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",3.5,"Пасс (1), Груз (1)",Нет,https://www.cian.ru/rent/flat/271634126


In [6]:
# 3. проверяем нет ли дублей, если есть удаляем
df.drop_duplicates(inplace=True)
df.shape

(23368, 23)

In [7]:
# 4."Количество комнат" заполняем отсутствующие данные медианой по общей площади
# преобразуем столбец 'Количество комнат' в числовой тип, заменяя некорректные значения на NaN
df['Количество комнат'] = pd.to_numeric(df['Количество комнат'], errors='coerce')
# создаём два новых столбца: 'Общая площадь, м2' и 'Жилая площадь, м2'
def extract_areas(area_str):
    try:
        parts = area_str.split('/')
        total_area = float(parts[0]) if len(parts) > 0 else None
        living_area = float(parts[1]) if len(parts) > 1 else None
        return pd.Series([total_area, living_area])
    except:
        return pd.Series([None, None])
df[['Общая площадь, м2', 'Жилая площадь, м2']] = df['Площадь, м2'].apply(extract_areas)
# группируем по 'Общая площадь, м2' и вычисляем медиану для каждой группы
medians = df.groupby('Общая площадь, м2')['Количество комнат'].transform('median')
# заменяем NaN значения в 'Количество комнат' на медиану соответствующей группы
df['Количество комнат'] = df['Количество комнат'].fillna(medians)
# чтобы дробных чисел не было
df['Количество комнат'] = df['Количество комнат'].round().astype('Int64')
# Заполним NA медианой по всему столбцу (после округления)
global_med = df['Количество комнат'].median()  # это может быть float или Int64
df['Количество комнат'] = df['Количество комнат'].fillna(global_med).astype('Int64')


In [8]:
# отсутствующие значения "Метро" заменим на 'unknown'
# добавим столбец метро без времени до него
df['Станция Метро'] = df['Метро'].str.split('(').str[0].str.strip()
# заменяем  NaN на 'unknown'
df['Станция Метро'] = df['Станция Метро'].fillna('unknown')
# заменяем строки, которые пустые или содержат только пробелы на 'unknown'
df['Станция Метро'] = df['Станция Метро'].replace(r'^\s*$', 'unknown', regex=True)

In [9]:
df['Парковка'].unique()


array(['подземная', nan, 'наземная', 'открытая', 'многоуровневая',
       'на крыше'], dtype=object)

In [10]:
# "Парковка" удаляем выбросы заполняем моду или может 'unknown'
# Проверяем количество пропущенных значений в столбце 'Парковка'
missing_values_count = df['Парковка'].isna().sum()
print(missing_values_count)


13417


In [11]:
df.isna().sum()

Unnamed: 0                      0
ID  объявления                  0
Количество комнат               0
Тип                             0
Метро                        1315
Адрес                           0
Площадь, м2                     0
Дом                             0
Парковка                    13417
Цена                            0
Телефоны                        0
Описание                        0
Ремонт                       2755
Площадь комнат, м2           8910
Балкон                       7978
Окна                         6613
Санузел                      2672
Можно с детьми/животными     6096
Дополнительно                 357
Высота потолков, м          12162
Лифт                         5500
Мусоропровод                10522
Ссылка на объявление            0
Общая площадь, м2               0
Жилая площадь, м2            3060
Станция Метро                   0
dtype: int64

In [12]:
df.shape

(23368, 26)

In [13]:
#рассчет среднего значения без выбросов по высоте потолков
df_mean = df['Высота потолков, м'].mean()
df_std = df['Высота потолков, м'].std()
df_filtered = df[(df['Высота потолков, м']> df_mean - 2*df_std) & (df['Высота потолков, м']<df_mean + 2*df_std)]
df_filtered['Высота потолков, м'].mean

<bound method Series.mean of 0        3.00
1        3.50
2        3.20
3        3.20
4        3.90
         ... 
23356    2.50
23357    2.65
23360    2.64
23361    2.64
23366    2.65
Name: Высота потолков, м, Length: 11189, dtype: float64>

In [14]:
# Заполняем пустые значения в столбце "Можно с детьми/животными" 'unknown'
df['Можно с детьми/животными'] = df['Можно с детьми/животными'].fillna('unknown') 
# Заполняем пустые значения в столбце "Дополнительно" 'unknown'
df['Дополнительно'] = df['Дополнительно'].fillna('unknown')
# Заполняем пустые значения в столбце "Лифт" 'unknown'
df['Лифт'] = df['Лифт'].fillna('unknown')
# Заполняем пустые значения в столбце "Мусоропровод" 'unknown'
df['Мусоропровод'] = df['Мусоропровод'].fillna('unknown')
# Заполняем пустые значения в столбце "Высотка потолков" средним значением по датафрейму с удаленными выбросами(3.00)
df['Высота потолков, м'] = df['Высота потолков, м'].fillna(3.00)

In [15]:
#оставшиеся категориальные признаки меняем на моду
#оставшиеся численные признаки меняем на медиану
missed_values = df.isna().sum()
missed_values = missed_values[missed_values > 0]
df_filled = df[missed_values.index].copy()
for col in df_filled.columns:
    if df_filled[col].dtype == 'object': # Категориальные признаки
        df_filled[col] = df_filled[col].fillna(df_filled[col].mode()[0])
    else: # Численные признаки
        df_filled[col] = df_filled[col].fillna(df_filled[col].median())

In [16]:
df3 = pd.concat([df,df_filled], axis=1)

In [ ]:
# Фильтруем по адресу с упоминанием "Москва"
moscow_flats = df[df['address'].str.contains('Москва', case=False, na=False)]

# Считаем количество
num_moscow_flats = len(moscow_flats)

print(f"Количество квартир в Москве: {num_moscow_flats}")


In [17]:
# заменяем NaN на False
df['Метро'].fillna(False, inplace=True)
# "Количество комнат" удаляем выбросы заполняем модой по общей площади 
df['Парковка'].fillna(df['Парковка'].mode()[0], inplace=True)
# "Количество комнат" удаляем выбросы заполняем модой по общей площади 
df['Ремонт'].fillna(df['Ремонт'].mode()[0], inplace=True)
# "Количество комнат" удаляем выбросы заполняем модой по общей площади 
df['Окна'].fillna(df['Окна'].mode()[0], inplace=True)
# "Количество комнат" удаляем выбросы заполняем модой по общей площади 
df['Санузел'].fillna(df['Санузел'].mode()[0], inplace=True)
# заменяем NaN на False
df['Балкон'].fillna(False, inplace=True)
# заменяем NaN на False
df['Можно с детьми/животными'].fillna(False, inplace=True)
# заменяем NaN на False
df['Дополнительно'].fillna(False, inplace=True)
# "Количество комнат" удаляем выбросы заполняем медианой по общей площади 
df['Высота потолков, м'].fillna(df['Высота потолков, м'].median(), inplace=True)
# заменяем NaN на False
df['Лифт'].fillna(False, inplace=True)
# заменяем NaN на False
df['Мусоропровод'].fillna(False, inplace=True)


/tmp/ipykernel_7509/1354217526.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Метро'].fillna(False, inplace=True)
/tmp/ipykernel_7509/1354217526.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df